# Time Series Forecasting

```
pip install darts pandas plotly
```

In [ ]:
import darts
import pandas as pd
import plotly.graph_objects as go
from darts.datasets import AirPassengersDataset
from darts.metrics import mae
from darts.models import NaiveMean, NaiveSeasonal
from darts.utils.statistics import (
    check_seasonality,
    extract_trend_and_seasonality,
    stationarity_test_adf,
)
from plotly.express import line
from plotly.subplots import make_subplots

In [ ]:
series: darts.TimeSeries = AirPassengersDataset().load()
dataframe: pd.DataFrame = series.to_dataframe(backend="pandas", time_as_index=False)
print(dataframe)  # noqa: T201


In [ ]:
line(
    x=dataframe["Month"],
    y=dataframe["#Passengers"],
    width=800,
    height=400,
    labels={"x": "Month", "y": "Passengers"},
    title="Air Passengers Over Time",
)

In [ ]:
check_seasonality(series)

Am I really going to believe whatever the computer says? Yes, yes I will.

##### Decomposing a time series

$y(t) = Trend + Seasonality + Noise$


$y(t) = Trend * Seasonality * Noise$

In [ ]:
trend: darts.TimeSeries
seasonality: darts.TimeSeries
trend, seasonality = extract_trend_and_seasonality(series)
trend_df = trend.to_dataframe(backend="pandas", time_as_index=False)
seasonality_df = seasonality.to_dataframe(backend="pandas", time_as_index=False)

In [ ]:
fig = make_subplots(rows=1, cols=2)

fig.add_trace(
    go.Scatter(x=trend_df["Month"], y=trend_df["0"]),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(x=seasonality_df["Month"], y=seasonality_df["0"]),
    row=1,
    col=2,
)

fig.update_layout(height=400, title_text="Trend and Seasonality")
fig.show()


In [ ]:
stationarity_test_adf(series)

In [ ]:
fig = make_subplots(rows=1, cols=1)

stationary_series = series / trend
stat_df = stationary_series.to_dataframe(backend="pandas", time_as_index=False)

fig.add_trace(
    go.Scatter(x=stat_df["Month"], y=stat_df["#Passengers"]),
    row=1,
    col=1,
)
fig.show()

In [ ]:
stationarity_test_adf(stationary_series)


### Split the data

In [ ]:
train: darts.TimeSeries
test: darts.TimeSeries
train, test = series.split_before(pd.Timestamp(year=1958, month=1, day=1))
train_df = train.to_dataframe(backend="pandas", time_as_index=False)
test_df = test.to_dataframe(backend="pandas", time_as_index=False)
fig2 = make_subplots(rows=1, cols=1)

fig2.add_trace(
    go.Scatter(
        x=train_df["Month"],
        y=train_df["#Passengers"],
        name="Train",
    ),
)
fig2.add_trace(
    go.Scatter(
        x=test_df["Month"],
        y=test_df["#Passengers"],
        name="Test",
    ),
)

fig2.update_layout(height=400, title_text="Trend and Seasonality")
fig2.show()


### Models

Let's start with a naive approach

In [ ]:
naive_mean_model = NaiveMean()
naive_mean_model.fit(train)
naive_mean_forecast = naive_mean_model.predict(36)

forecast_df = naive_mean_forecast.to_dataframe(backend="pandas", time_as_index=False)
fig3 = make_subplots(rows=1, cols=1)

fig3.add_trace(
    go.Scatter(
        x=train_df["Month"],
        y=train_df["#Passengers"],
        name="Train",
    ),
)
fig3.add_trace(
    go.Scatter(
        x=test_df["Month"],
        y=test_df["#Passengers"],
        name="Test",
    ),
)
fig3.add_trace(
    go.Scatter(
        x=forecast_df["Month"],
        y=forecast_df["#Passengers"],
        name="Forecast",
    ),
)

fig3.update_layout(height=400, title_text="Trend and Seasonality")
fig3.show()


In [ ]:
naive_seasonal_model = NaiveSeasonal(K=12)
naive_seasonal_model.fit(train)
naive_seasonal_forecast = naive_seasonal_model.predict(36)

forecast_df = naive_seasonal_forecast.to_dataframe(
    backend="pandas",
    time_as_index=False,
)
fig4 = make_subplots(rows=1, cols=1)

fig4.add_trace(
    go.Scatter(
        x=train_df["Month"],
        y=train_df["#Passengers"],
        name="Train",
    ),
)
fig4.add_trace(
    go.Scatter(
        x=test_df["Month"],
        y=test_df["#Passengers"],
        name="Test",
    ),
)
fig4.add_trace(
    go.Scatter(
        x=forecast_df["Month"],
        y=forecast_df["#Passengers"],
        name="Forecast",
    ),
)

fig4.update_layout(height=400, title_text="Trend and Seasonality")
fig4.show()


In [ ]:
from darts.models import RegressionEnsembleModel

ensemble_model = RegressionEnsembleModel(
    forecasting_models=[
        naive_mean_model.untrained_model(),
        naive_seasonal_model.untrained_model(),
    ],
    regression_train_n_points=36,
)
ensemble_model.fit(train)
predictions = ensemble_model.predict(36)

forecast_df = predictions.to_dataframe(
    backend="pandas",
    time_as_index=False,
)
fig5 = make_subplots(rows=1, cols=1)

fig5.add_trace(
    go.Scatter(
        x=train_df["Month"],
        y=train_df["#Passengers"],
        name="Train",
    ),
)
fig5.add_trace(
    go.Scatter(
        x=test_df["Month"],
        y=test_df["#Passengers"],
        name="Test",
    ),
)
fig5.add_trace(
    go.Scatter(
        x=forecast_df["Month"],
        y=forecast_df["#Passengers"],
        name="Forecast",
    ),
)

fig5.update_layout(height=400, title_text="Trend and Seasonality")
fig5.show()


##### Probabilistic models

In [ ]:
from darts.models import ExponentialSmoothing

es_model = ExponentialSmoothing()
es_model.fit(train)
predictions = es_model.predict(36, num_samples=100)

forecast_df = predictions.quantiles_df(quantiles=(0.1, 0.5, 0.9))
forecast_df = forecast_df.reset_index()

fig6 = make_subplots(rows=1, cols=1)

fig6.add_trace(
    go.Scatter(
        x=train_df["Month"],
        y=train_df["#Passengers"],
        name="Train",
    ),
)
fig6.add_trace(
    go.Scatter(
        x=test_df["Month"],
        y=test_df["#Passengers"],
        name="Test",
    ),
)
fig6.add_trace(
    go.Scatter(
        x=forecast_df["Month"],
        y=forecast_df["#Passengers_0.5"],
        name="Forecast Mean",
    ),
)
fig6.add_trace(
    go.Scatter(
        x=forecast_df["Month"],
        y=forecast_df["#Passengers_0.1"],
        name="Forecast 0.1",
        line_color="rgba(0,0,0,0)",
        showlegend=False,
    ),
)
fig6.add_trace(
    go.Scatter(
        x=forecast_df["Month"],
        y=forecast_df["#Passengers_0.9"],
        fill="tonexty",
        name="Forecast 0.9",
        fillcolor="rgba(231,107,243,0.3)",
        line_color="rgba(0,0,0,0)",
        showlegend=False,
    ),
)

fig6.update_layout(height=400, title_text="Trend and Seasonality")
fig6.show()


Okay but how wrong is it?

In [ ]:
naive_mean_mae = mae(series, naive_mean_forecast)
naive_seasonal_mae = mae(series, naive_seasonal_forecast)
print(f"MAE for the naive mean: {naive_mean_mae}")  # noqa: T201
print(f"MAE for the naive seasonal: {naive_seasonal_mae}")  # noqa: T201


### Trying other models

In [ ]:
from darts.models import AutoARIMA, ExponentialSmoothing, Theta
from darts.models.forecasting.forecasting_model import ForecastingModel


def eval_model(
    model: ForecastingModel,
    training_data: darts.TimeSeries,
    test_data: darts.TimeSeries,
) -> None:
    """Evaluate a darts model on the training data and print the MAE.

    Args:
    ----
    model: An untrained darts model.
    training_data: A list of training data.
    test_data: A list of test data.

    """
    model.fit(training_data)
    forecast = model.predict(len(test_data))
    print(f"Model {model} obtains MAE: {mae(test_data, forecast):.2f}")  # noqa: T201


eval_model(ExponentialSmoothing(), train, test)
eval_model(AutoARIMA(), train, test)
eval_model(Theta(), train, test)
